In [ ]:
!unzip /content/colord_balls.v1i.yolov8.zip

In [5]:
import os
import shutil
import random

# --- CONFIGURATION ---
# The source of your initial un-split data (from your zip)
source_images_dir = '/content/train/images'
source_labels_dir = '/content/train/labels'

# Destination paths (inside the /content/train folder)
dataset_root = '/content/train' # Use the existing 'train' folder as the new dataset root

# Define new destination directories that we will create within /content/train
val_images_dir = os.path.join(dataset_root, 'valid/images')
test_images_dir = os.path.join(dataset_root, 'test/images')
val_labels_dir = os.path.join(dataset_root, 'valid/labels')
test_labels_dir = os.path.join(dataset_root, 'test/labels')
train_labels_dir = os.path.join(dataset_root, 'images') # Labels remain in /content/train/images

# Define split ratios
train_ratio = 0.8  # Keep 80% in 'train'
val_ratio = 0.1
test_ratio = 0.1

# Class remapping setup
# Original names: 0:'Ball', 1:'green_ball', 2:'pink_ball', 3:'yellow_ball'
old_class_id_to_remove = 0
class_mapping = {
    1: 0, # old green_ball (ID 1) becomes new ID 0
    2: 1, # old pink_ball (ID 2) becomes new ID 1
    3: 2  # old yellow_ball (ID 3) becomes new ID 2
}

# --- DIRECTORY SETUP ---
for directory in [val_images_dir, test_images_dir, val_labels_dir, test_labels_dir]:
    os.makedirs(directory, exist_ok=True)
print("Destination directories created/verified.")

# --- DATA SPLITTING ---
all_files = os.listdir(source_images_dir)
all_images = [f for f in all_files if f.endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(all_images)

num_images = len(all_images)
num_train_to_move = int(train_ratio * num_images) # Calculate how many should REMAIN in train
num_val = int(val_ratio * num_images)

# Files to move out of the 'train' folder
val_images = all_images[num_train_to_move : num_train_to_move + num_val]
test_images = all_images[num_train_to_move + num_val :]

# Function to move files (will also process and clean labels)
def move_and_process_files(file_list, source_img_dir, source_lbl_dir, dest_img_dir, dest_lbl_dir):
    for filename in file_list:
        base_name = os.path.splitext(filename)[0]
        source_img_path = os.path.join(source_img_dir, filename)
        source_lbl_path = os.path.join(source_lbl_dir, base_name + '.txt')
        dest_img_path = os.path.join(dest_img_dir, filename)
        dest_lbl_path = os.path.join(dest_lbl_dir, base_name + '.txt')

        if os.path.exists(source_lbl_path):
            shutil.move(source_img_path, dest_img_path)

            # --- LABEL PROCESSING ---
            new_lines = []
            with open(source_lbl_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) > 0:
                        class_id = int(parts[0])
                        if class_id == old_class_id_to_remove:
                            continue  # Skip 'Ball' class
                        elif class_id in class_mapping:
                            parts[0] = str(class_mapping[class_id])
                            new_lines.append(' '.join(parts) + '\n')

            # Write the cleaned and re-mapped annotations to the new location
            with open(dest_lbl_path, 'w') as f:
                f.writelines(new_lines)

            # Delete the old label file from the source_labels_dir
            os.remove(source_lbl_path)

        else:
            print(f"Warning: Label file not found for {filename}. Skipping.")
            # Move the image back to its original location if the label is missing
            shutil.move(source_img_path, os.path.join(source_img_dir, filename))


# Split and clean the Val and Test sets
move_and_process_files(val_images, source_images_dir, source_labels_dir, val_images_dir, val_labels_dir)
move_and_process_files(test_images, source_images_dir, source_labels_dir, test_images_dir, test_labels_dir)

print("\nDataset split, moved, and cleaned completed!")

# --- CLEAN UP REMAINING TRAINING LABELS ---
# The remaining files in the original folders are the training set.
# Now, we only need to process their labels.
print("\nProcessing remaining training labels...")
def process_training_labels(label_dir):
    for filename in os.listdir(label_dir):
        if filename.endswith('.txt'):
            filepath = os.path.join(label_dir, filename)
            new_lines = []

            with open(filepath, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) > 0:
                        class_id = int(parts[0])
                        if class_id == old_class_id_to_remove:
                            continue  # Skip 'Ball' class
                        elif class_id in class_mapping:
                            parts[0] = str(class_mapping[class_id])
                            new_lines.append(' '.join(parts) + '\n')

            with open(filepath, 'w') as f:
                f.writelines(new_lines)

process_training_labels(source_labels_dir)
print("Training labels cleaned successfully!")

Destination directories created/verified.

Dataset split, moved, and cleaned completed!

Processing remaining training labels...
Training labels cleaned successfully!


In [6]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.0 MB/s eta 0:00:00


In [7]:
from ultralytics import YOLO

# Load the base YOLO model (use the correct name 'yolov8n.pt')
model = YOLO("yolov8n.pt")

# Train the model using the updated data.yaml
model.train(data="data.yaml", epochs=100)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, l

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x788809b647d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [ ]:
# Create the zip file first (same as Step 1 above)
!zip -r trainfinal_results.zip /content/runs/detect/train